In [1]:
pip install opencv-python numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2
import numpy as np
import time
import os

In [3]:
# Define the video path
video_path = "E:\AI Assign\AI Assignment video.mp4"

# Check if the file exists
if not os.path.exists(video_path):
    raise ValueError(f"Error: The video file {video_path} does not exist.")

In [4]:
# Try opening the video file
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise ValueError("Error: Unable to open video file.")

In [5]:
 # Read the first frame to get width and height
ret, frame = cap.read()
if ret:
    height, width = frame.shape[:2]
    # Define new dimensions (e.g., half the original size)
    new_width = width // 2
    new_height = height // 2
    # Initialize VideoWriter with the new dimensions
    out = cv2.VideoWriter('processed_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (new_width, new_height))
else:
    cap.release()
    raise ValueError("Error: Unable to read the first frame of the video file.")

In [6]:
# Define the quadrants (assuming 2x2 grid for simplicity)
def get_quadrant(x, y, width, height):
    if x < width / 2 and y < height / 2:
        return 1
    elif x >= width / 2 and y < height / 2:
        return 2
    elif x < width / 2 and y >= height / 2:
        return 3
    else:
        return 4

In [7]:
# Define color ranges for detection (BGR format)
color_ranges = {
    'red': ([0, 0, 100], [50, 56, 255]),
    'green': ([0, 100, 0], [50, 255, 50]),
    'blue': ([100, 0, 0], [255, 50, 50])
}

def detect_balls(frame):
    balls = []
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    for color, (lower, upper) in color_ranges.items():
        lower = np.array(lower, dtype="uint8")
        upper = np.array(upper, dtype="uint8")
        mask = cv2.inRange(hsv, lower, upper)
        cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
        for c in cnts:
            ((x, y), radius) = cv2.minEnclosingCircle(c)
            if radius > 10:  # Minimum size of ball
                balls.append((int(x), int(y), color))
    return balls

# Initialize variables
prev_positions = {}
event_log = []
start_time = time.time()

In [8]:
# Reset the capture to start from the beginning
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame to the new dimensions
    frame = cv2.resize(frame, (new_width, new_height))

    balls = detect_balls(frame)
    
    for (x, y, color) in balls:
        quadrant = get_quadrant(x, y, new_width, new_height)
        if color in prev_positions:
            prev_quadrant = prev_positions[color]
            if prev_quadrant != quadrant:
                timestamp = time.time() - start_time
                event_log.append((timestamp, prev_quadrant, color, 'Exit'))
                event_log.append((timestamp, quadrant, color, 'Entry'))
                cv2.putText(frame, f"Exit {prev_quadrant} - {timestamp:.2f}", (x, y - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                cv2.putText(frame, f"Entry {quadrant} - {timestamp:.2f}", (x, y - 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        prev_positions[color] = quadrant
        
        # Draw ball and quadrant
        cv2.circle(frame, (x, y), 10, (0, 255, 255), -1)
        cv2.putText(frame, f'{color} - {quadrant}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    
    out.write(frame)

cap.release()
out.release()

In [9]:
# Save event data to a text file
with open('event_log.txt', 'w') as f:
    for event in event_log:
        f.write(f"{event[0]:.2f}, {event[1]}, {event[2]}, {event[3]}\n")